In [ ]:
from decimal import Decimal

import pandas as pd

from nautilus_trader.core.datetime import dt_to_unix_nanos
from nautilus_trader.persistence.catalog import DataCatalog
from nautilus_trader.backtest.config import BacktestRunConfig, BacktestVenueConfig, BacktestDataConfig, BacktestEngineConfig
from nautilus_trader.backtest.node import BacktestNode
from nautilus_trader.model.data.tick import QuoteTick
from nautilus_trader.trading.config import ImportableStrategyConfig
from nautilus_trader.examples.strategies.ema_cross import EMACross, EMACrossConfig

In [ ]:
catalog = DataCatalog.from_env()

In [ ]:
catalog.instruments()
start = dt_to_unix_nanos(pd.Timestamp('2020-01-01', tz='UTC'))
end =  dt_to_unix_nanos(pd.Timestamp('2020-01-02', tz='UTC'))

catalog.quote_ticks(start=start, end=end)

In [ ]:

# Create a `base` config object to be shared with all backtests
base = BacktestRunConfig(
    venues=[
        BacktestVenueConfig(
            name="SIM",
            oms_type="HEDGING",
            account_type="MARGIN",
            base_currency="USD",
            starting_balances=["1000000 USD"],
        )
    ]
)
base

In [ ]:
instrument = catalog.instruments(as_nautilus=True)[0]

data_config=[
    BacktestDataConfig(
        catalog_path=str(DataCatalog.from_env().path),
        data_cls=QuoteTick,
        instrument_id=instrument.id.value,
        start_time=1580398089820000000,
        end_time=1580504394501000000,
    )
]

config = base.update(
    data=data_config,
    engine=BacktestEngineConfig()
)

config

In [ ]:
PARAM_SET = [
    {"fast_ema": 5, "slow_ema": 20},
    {"fast_ema": 10, "slow_ema": 50},
    {"fast_ema": 30, "slow_ema": 100},
]

configs = []
for params in PARAM_SET:
    strategies = [
        ImportableStrategyConfig(
            path=EMACross.fully_qualified_name(),
            config=EMACrossConfig(
                instrument_id=instrument.id.value,
                bar_type='EUR/USD.SIM-15-MINUTE-BID-INTERNAL',
                trade_size=Decimal(1_000_000),
                **params
            ),
        ),
    ]
    # Create the final config
    new = config.replace(strategies=strategies)
    
    configs.append(new)

In [ ]:
print("\n\n".join(map(str, configs)))

In [ ]:
node = BacktestNode()

# Single threaded synchronous run

In [ ]:
results = node.run_sync(run_configs=configs)

In [ ]:
pd.DataFrame([r.stats_pnls for r in results])['USD'].apply(pd.Series)

# Run a parallel/distributed backtest

In [ ]:
%pip install distributed

In [ ]:
task = node.build_graph(run_configs=configs)
task

In [ ]:
# Create a local dask client - not a requirement, but allows parallelising the runs
from distributed import Client
client = Client(n_workers=3)
client

In [ ]:
results = task.compute()

In [ ]:
r = results[0]

In [ ]:
r.stats_pnls